In [25]:
path_dir1 = 'Data2/ECG2020/WFDB_ChapmanShaoxing'
path_dir2 = 'Data2/ECG2020/WFDB_CPSC2018'
path_dir3 = 'Data2/ECG2020/WFDB_CPSC2018_2'
path_dir4 = 'Data2/ECG2020/WFDB_Ga'
# path_dir5 = 'Data2/ECG2020/WFDB_Ningbo'
path_dir6 = 'Data2/ECG2020/WFDB_PTB'
path_dir7 = 'Data2/ECG2020/WFDB_PTBXL'
path_dir8 = 'Data2/ECG2020/WFDB_StPetersburg'


file_list1 = os.listdir(path_dir1)
file_list2 = os.listdir(path_dir2)
file_list3 = os.listdir(path_dir3)
file_list4 = os.listdir(path_dir4)
# file_list5 = os.listdir(path_dir5)
file_list6 = os.listdir(path_dir6)
file_list7 = os.listdir(path_dir7)
file_list8 = os.listdir(path_dir8)




file_list1.sort()
file_list2.sort() 
file_list3.sort() 
file_list4.sort() 
# file_list5.sort()
file_list6.sort() 
file_list7.sort()
file_list8.sort()

In [27]:
file_list1[0]

'JS00001.hea'

In [17]:
import os
import numpy as np, sys,os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.io import loadmat
import wfdb
import tarfile
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedKFold
from keras.preprocessing.sequence import pad_sequences
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
import tensorflow_addons as tfa
import tensorflow as tf
from tensorflow import keras
#from keras.utils import plot_model
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import butter, lfilter, filtfilt
from scipy.signal import find_peaks
from scipy.signal import peak_widths
from scipy.signal import savgol_filter

### 데이터 임포트를 위한 사용자 정의 함수 (데이터 전처리)

In [59]:
def load_challenge_data(filename):  # file 의 전체 경로가 input
    x = loadmat(filename)
    data = np.asarray(x['val'], dtype=np.float64)
    new_file = filename.replace('.mat','.hea')
    input_header_file = os.path.join(new_file)
    with open(input_header_file,'r') as f:
        header_data=f.readlines()
    return data, header_data


def clean_up_gender_data(gender):  # pre-processing SEX
    gender = np.asarray(gender)
    gender[np.where(gender == "Male")] = 0
    gender[np.where(gender == "male")] = 0
    gender[np.where(gender == "M")] = 0
    gender[np.where(gender == "Female")] = 1
    gender[np.where(gender == "female")] = 1
    gender[np.where(gender == "F")] = 1
    gender[np.where(gender == "NaN")] = 2
    np.unique(gender)
    gender = gender.astype(np.int)
    return gender

def clean_up_age_data(age):  # pre-processing Age
    age = np.asarray(age)
    age[np.where(age == "NaN")] = -1
    np.unique(age)
    age = age.astype(np.int)
    return age


def import_gender_and_age(age, gender):  #age, Sex data import   , import_key_data를 거친 age와 gender를 input으로 넣어줌
    gender_binary = clean_up_gender_data(gender)
    age_clean = clean_up_age_data(age)
    print("gender data shape: {}".format(gender_binary.shape[0]))
    print("age data shape: {}".format(age_clean.shape[0]))
    return age_clean, gender_binary

def import_key_data(path):  #data import
    gender=[]
    age=[]
    labels=[]
    ecg_filenames=[]
    for subdir, dirs, files in sorted(os.walk(path)):
        for filename in files:
            filepath = subdir + os.sep + filename
            if filepath.endswith(".mat"):
                data, header_data = load_challenge_data(filepath)
                labels.append(header_data[15][5:-1])  #header list에서 label 가져와서 리스트에 넣기
                ecg_filenames.append(filepath)  #file path 저장
                gender.append(header_data[14][6:-1]) #header list에서 Sex 가져와서 리스트에 넣기
                age.append(header_data[13][6:-1])  #header list에서 age 가져와서 리스트에 넣기
    return gender, age, labels, ecg_filenames


def get_signal_lengths(path, title):  #signal length 얻기 size : 5000
    signal_length=[]
    for subdir, dirs, files in sorted(os.walk(path)): #디렉토리 내의 파일을 iterator로 반환
        for filename in files:
            filepath = subdir + os.sep + filename
            if filepath.endswith(".mat"):
                data, header_data = load_challenge_data(filepath)
                splitted = header_data[0].split()
                signal_length.append(splitted[3])
    signal_length_df = pd.DataFrame(signal_length)
    signal_count=signal_length_df[0].value_counts()
    plt.figure(figsize=(20,10))
    plt.title(title,fontsize =36)
    sns.barplot(signal_count[:10,].index, signal_count[:10,].values)
    
    
def make_undefined_class(labels, df_unscored):   # 아직 기능을 잘 모르겠음
    df_labels = pd.DataFrame(labels)
    for i in range(len(df_unscored.iloc[0:,1])):
        df_labels.replace(to_replace=str(df_unscored.iloc[i,1]), inplace=True ,value="undefined class", regex=True)

    '''
    #equivalent classes
    codes_to_replace=['713427006','284470004','427172004']
    replace_with = ['59118001','63593006','17338001']

    for i in range(len(codes_to_replace)):
        df_labels.replace(to_replace=codes_to_replace[i], inplace=True ,value=replace_with[i], regex=True)
    '''
    return df_labels


def onehot_encode(df_labels):
    one_hot = MultiLabelBinarizer()
    y=one_hot.fit_transform(df_labels[0].str.split(pat=','))
    print("The classes we will look at are encoded as SNOMED CT codes:")
    print(one_hot.classes_)
    y = np.delete(y, -1, axis=1)
    print("classes: {}".format(y.shape[1]))
    return y, one_hot.classes_[0:-1]

In [60]:
data , header_data = load_challenge_data(os.path.join('Data2/ECG2020/WFDB_ChapmanShaoxing',file_list1[1]))

In [38]:
len(header_data)  #header_file 길이

19

In [62]:
import_key_data('Data2/ECG2020/WFDB_ChapmanShaoxing')

(['Female',
  'Male',
  'Male',
  'Female',
  'Female',
  'Male',
  'Female',
  'Male',
  'Male',
  'Male',
  'Male',
  'Male',
  'Male',
  'Male',
  'Male',
  'Male',
  'Male',
  'Female',
  'Male',
  'Female',
  'Female',
  'Male',
  'Female',
  'Female',
  'Female',
  'Female',
  'Male',
  'Male',
  'Female',
  'Female',
  'Male',
  'Female',
  'Male',
  'Female',
  'Male',
  'Female',
  'Female',
  'Female',
  'Female',
  'Male',
  'Female',
  'Male',
  'Male',
  'Male',
  'Female',
  'Male',
  'Male',
  'Female',
  'Female',
  'Female',
  'Female',
  'Female',
  'Female',
  'Male',
  'Female',
  'Female',
  'Male',
  'Female',
  'Male',
  'Female',
  'Male',
  'Female',
  'Female',
  'Male',
  'Male',
  'Female',
  'Female',
  'Male',
  'Male',
  'Male',
  'Male',
  'Male',
  'Male',
  'Male',
  'Male',
  'Female',
  'Female',
  'Male',
  'Female',
  'Female',
  'Female',
  'Male',
  'Female',
  'Male',
  'Male',
  'Female',
  'Male',
  'Female',
  'Female',
  'Female',
  'Male',


<generator object walk at 0x7fcf13424bd0>